In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from itertools import combinations
import itertools


In [4]:
csfbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\CSF_Biomarker1\csfbiomk1_20231117.csv"
csfbiomk1 = pd.read_csv(csfbiomk1_file_path)
csfbiomk1 = pd.DataFrame(csfbiomk1)

# right_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Volumetric_Analysis\rh_aparc_volume_20231117.csv"
# right_volume = pd.read_csv(right_volume_file_path)
# right_volume = pd.DataFrame(right_volume)


In [5]:
csfbiomk1.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,24.4,0.0,0.0,0.0,598.0,373.96,8706.69,487.34,0.6059,1.0
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print("Column Names:")
print(csfbiomk1.columns[2])

Column Names:
checkin_bin


In [7]:
#group them base on the value in the third column which indicates their level of playing
csfbiomk1_grouped = csfbiomk1.groupby(csfbiomk1.iloc[:, 2])

NFL_csfbiomk1_grouped = pd.DataFrame()
CP_csfbiomk1_grouped = pd.DataFrame()
HC_csfbiomk1_grouped = pd.DataFrame()


# group_name : 1, 2, 3   group_data: 
for group_name, group_data in csfbiomk1_grouped:
    if group_name == 1:
        NFL_csfbiomk1_grouped = pd.concat([NFL_csfbiomk1_grouped,group_data], ignore_index = True)
    if group_name == 2:
        CP_csfbiomk1_grouped = pd.concat([CP_csfbiomk1_grouped,group_data], ignore_index = True)
    if group_name == 3:
        HC_csfbiomk1_grouped = pd.concat([HC_csfbiomk1_grouped,group_data], ignore_index = True)
    
#print("DataFrame for NFL:")
#print(NFL_right_grouped.head())
NFL_csfbiomk1_grouped.head()



,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
0,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,1,1,2,3,15.0,22.0,8220.9,5421.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9774,1.0
2,1011,1,1,1,2,5,16.0,20.0,9307.0,9307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,1,1,1,3,19.0,17.0,9866.7,6173.3,...,24.0,0.0,0.0,0.0,174.0,472.76,NaN,NaN,NaN,3.0
4,1018,1,1,1,1,3,16.0,23.0,10635.9,7929.6,...,20.5,0.0,0.0,0.0,202.0,336.18,7297.61,316.56,2.0394,1.0


In [8]:
index_of_chiiyrs_pg = NFL_csfbiomk1_grouped.columns.get_loc("chiiyrs_pg")
NFL_csfbiomk1_grouped.drop(columns=NFL_csfbiomk1_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
CP_csfbiomk1_grouped.drop(columns=CP_csfbiomk1_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
HC_csfbiomk1_grouped.drop(columns=HC_csfbiomk1_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)

NFL_csfbiomk1_grouped.head()


,subject_id,checkin_bin,c_strem2_FLAG_insufficient,c_PDGFRbeta_FLAG_insufficient,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9774,1.0
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,0.0,0.0,1446.0,333.0,6889.0,670.0,24.0,0.0,0.0,0.0,174.0,472.76,NaN,NaN,NaN,3.0
4,1018,1,0.0,0.0,1563.0,209.0,5360.0,463.0,20.5,0.0,0.0,0.0,202.0,336.18,7297.61,316.56,2.0394,1.0


In [9]:
#combine all three classes
combined_csfbiomk1 = pd.concat([NFL_csfbiomk1_grouped, CP_csfbiomk1_grouped, HC_csfbiomk1_grouped], ignore_index=True)

combined_csfbiomk1.head()

,subject_id,checkin_bin,c_strem2_FLAG_insufficient,c_PDGFRbeta_FLAG_insufficient,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9774,1.0
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,0.0,0.0,1446.0,333.0,6889.0,670.0,24.0,0.0,0.0,0.0,174.0,472.76,NaN,NaN,NaN,3.0
4,1018,1,0.0,0.0,1563.0,209.0,5360.0,463.0,20.5,0.0,0.0,0.0,202.0,336.18,7297.61,316.56,2.0394,1.0


In [10]:
# Remove NaNs
print("Size before:", combined_csfbiomk1.shape)
combined_csfbiomk1.dropna(inplace=True)
print("Size after:", combined_csfbiomk1.shape)

Size before: (236, 18)
Size after: (148, 18)


In [62]:
# Separate based on the level of professionalism
X = combined_csfbiomk1.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk1['checkin_bin']

In [63]:
X.head()

,c_strem2_FLAG_insufficient,c_PDGFRbeta_FLAG_insufficient,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
4,0.0,0.0,1563.0,209.0,5360.0,463.0,20.5,0.0,0.0,0.0,202.0,336.18,7297.61,316.56,2.0394,1.0
5,0.0,0.0,3328.0,967.0,21115.0,1500.0,104.2,0.0,0.0,0.0,828.0,600.31,12695.85,1218.89,5.2955,1.0
6,0.0,0.0,1595.0,358.0,9435.0,898.0,29.4,0.0,0.0,0.0,222.0,452.69,1504.20,492.71,1.3735,1.0
8,0.0,0.0,2646.0,569.0,15812.0,1332.0,74.0,0.0,0.0,0.0,630.0,747.93,12874.13,1169.94,6.9061,1.0
11,0.0,0.0,4210.0,385.0,11253.0,805.0,54.3,0.0,0.0,0.0,464.0,787.93,11141.23,820.31,3.1215,1.0


In [64]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [65]:
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [66]:
X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [39]:
# RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# store evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

Accuracy: 0.6, Precision: 0.6, Recall: 0.6, f1:0.6


## Try Bagging


In [22]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                  n_estimators=10, random_state=42)


bagging_model.fit(X_train, y_train)
    
y_pred_bagging = bagging_model.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.6, Precision: 0.6, Recall: 0.6, f1:0.6


In [28]:
y_test

201    3
85     1
225    3
34     1
169    2
20     1
118    1
57     1
131    2
16     1
50     1
159    2
230    3
105    1
217    3
103    1
33     1
107    1
198    3
56     1
54     1
170    2
64     1
192    3
91     1
40     1
101    1
211    3
132    2
18     1
Name: checkin_bin, dtype: int64

## Try XGBoost

In [ ]:
from xgboost import XGBClassifier


## Try XGBoost

In [49]:
X = combined_csfbiomk1.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk1['checkin_bin']

In [50]:
y

4      1
5      1
6      1
8      1
11     1
      ..
229    3
230    3
231    3
232    3
235    3
Name: checkin_bin, Length: 148, dtype: int64

In [51]:
# Since for XGBoost, the y values should start at 0, you need to do some justification
y = y-1

In [52]:
y

4      0
5      0
6      0
8      0
11     0
      ..
229    2
230    2
231    2
232    2
235    2
Name: checkin_bin, Length: 148, dtype: int64

In [53]:
# Do the splitting again to make sure that this adjustemnt is applied to your new X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [54]:
X_test

,c_strem2_FLAG_insufficient,c_PDGFRbeta_FLAG_insufficient,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_tTau_FLAG_below_ref_range,c_FLAG_hemolysis,c_tTau_FLAG_insufficient,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,dxcte_ptau217_csf_flag_dv
0,0.0,0.0,0.662833,0.289236,1.020198,1.449275,0.487152,0.0,1.0,0.0,0.271786,0.221119,0.150849,0.309082,0.317442,0.0
1,0.0,0.0,0.244487,0.190633,0.278316,0.363124,0.037473,0.0,0.0,0.0,0.004902,0.156840,0.195112,0.292607,0.097098,0.0
2,0.0,0.0,0.405764,0.230896,0.885679,1.080515,0.302998,0.0,0.0,0.0,0.118192,0.247274,0.326219,0.468445,0.110534,0.0
3,0.0,0.0,0.214856,0.091208,0.193588,0.249597,0.038544,0.0,0.0,0.0,0.146514,0.077653,0.123012,0.269597,0.037594,1.0
4,0.0,0.0,0.396699,0.152835,0.338500,0.412238,0.111349,0.0,0.0,0.0,0.092048,0.107950,0.201482,0.274610,0.090387,0.0
5,0.0,0.0,0.355026,0.119145,0.376725,0.465378,0.182013,0.0,0.0,0.0,0.110566,0.229553,0.405548,0.494594,0.230965,0.0
6,0.0,0.0,0.327425,0.178307,0.388175,0.435588,0.328694,0.0,0.0,0.0,0.125817,0.247274,0.661893,0.544178,0.469764,0.0
7,0.0,0.0,0.136382,0.122432,0.197698,0.247987,0.141328,0.0,0.0,0.0,0.320806,0.140887,0.120373,0.344074,0.259701,0.0
8,0.0,0.0,0.301718,0.169269,0.345899,0.424316,0.117773,0.0,0.0,0.0,0.131264,0.113096,-0.020543,0.235370,0.190641,0.0
9,0.0,0.0,0.324313,0.153657,0.533556,0.623188,0.299786,0.0,0.0,0.0,0.127996,0.171325,0.099701,0.245007,0.235373,0.0


In [55]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=100, random_state=42)


xgb_model.fit(X_train, y_train)
    
y_pred_xgb = xgb_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")



Accuracy: 0.5333333333333333, Precision: 0.5525210084033615, Recall: 0.5333333333333333, f1:0.5403907203907204


## Tuning Hyperparameters

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = RandomForestClassifier(**best_params)

best_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model.predict(X_test)

# evluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


Accuracy: 0.5, Precision: 0.5111111111111112, Recall: 0.5, F1 Score: 0.5046620046620046


In [71]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=7, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = RandomForestClassifier(**best_params)

best_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model.predict(X_test)

# evluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


Accuracy: 0.4888888888888889, Precision: 0.4971193415637859, Recall: 0.4888888888888889, F1 Score: 0.48564845782237087


In [72]:
y_pred2  = best_model.predict(X_train_scaled)

accuracy_train = accuracy_score(y_train, y_pred2)
precision_bagging = precision_score(y_train, y_pred2, average='weighted')
recall_bagging = recall_score(y_train, y_pred2, average='weighted')
f1_bagging = f1_score(y_train, y_pred2, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")
# print(f"Accuracy: {accuracy}")


Accuracy: 0.4888888888888889, Precision: 0.4971193415637859, Recall: 0.4888888888888889, f1:0.48564845782237087


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [70]:
y_train

149    2
227    3
173    2
4      1
113    1
      ..
110    1
171    2
27     1
148    2
167    2
Name: checkin_bin, Length: 103, dtype: int64

## Tuning Hyperparameters (on Bagging- Incorrect)


In [40]:
# from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV

# bagging_model = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100, random_state=42), 
#                                   n_estimators=10, random_state=42)

# base_model = DecisionTreeClassifier()
# # bagging_model = BaggingClassifier(base_model)

# # Hyperparameters
# param_grid = {
#     'base_estimator__max_depth': [3, 5, 7],
#     'n_estimators': [10, 50, 100],
#     'max_samples': [0.5, 0.7, 1.0],
#     'max_features': [0.5, 0.7, 1.0],
# }


# grid_search = GridSearchCV(bagging_model, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_

# best_bagging_model = BaggingClassifier(base_model, **best_params)
# best_bagging_model.fit(X_train, y_train)

   
# y_pred_bagging = bagging_model.predict(X_test)

# accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
# precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
# recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
# f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')

# print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")




C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

...

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

TypeError: BaggingClassifier.__init__() got an unexpected keyword argument 'base_estimator__max_depth'

In [59]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


base_model = DecisionTreeClassifier()
bagging_model = BaggingClassifier(base_model)

# Hyperparameters
param_grid = {
    'n_estimators': [10, 50, 100],
    'base_estimator__max_depth': [None, 5, 10],
}


grid_search = GridSearchCV(bagging_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_bagging_model = BaggingClassifier(base_model, n_estimators=best_params['n_estimators'], max_depth=best_params['base_estimator__max_depth'])
best_bagging_model.fit(X_train, y_train)
   
y_pred_bagging = bagging_model.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")




ValueError: Invalid parameter 'base_estimator_max_depth' for estimator BaggingClassifier(estimator=DecisionTreeClassifier()). Valid parameters are: ['base_estimator', 'bootstrap', 'bootstrap_features', 'estimator', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].